In [1]:
import panel as pn
pn.extension()

The ``FileDownload`` widget allows downloading a file on the frontend by sending the file data to the browser either on initialization (if ``embed=True``) or when the button is clicked.

For more information about listening to widget events and laying out widgets refer to the [widgets user guide](../../user_guide/Widgets.ipynb). Alternatively you can learn how to build GUIs by declaring parameters independently of any specific widgets in the [param user guide](../../user_guide/Param.ipynb). To express interactivity entirely using Javascript without the need for a Python server take a look at the [links user guide](../../user_guide/Param.ipynb).

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

##### Core

* **``auto``** (boolean):  Whether to download the file the initial click (if `True`) or when clicking a second time (or via the right-click Save file menu).
* **``callback``** (callable): A callable that returns a file or file-like object (takes precedence over `file` if set). 
* **``embed``** (boolean):  Whether to embed the data on initialization.
* **``file``** (str or file-like object):  A path to a file or a file-like object.
* **``filename``** (str): The filename to save the file as.

##### Display

* **``button_type``** (str): A button theme; should be one of ``'default'`` (white), ``'primary'`` (blue), ``'success'`` (green), ``'info'`` (yellow), or ``'danger'`` (red)
* **``label``** (str): A custom label for the download button (by default uses the filename)
* **``name``** (str): The title of the widget

___

The `FileDownload` widget accepts a path to a file or a file-like object (with a `.read` method) if the latter is provided a `filename` must also be set. By default (`auto=True` and `embed=False`) the file is only transferred to the browser after the button is clicked (this requires a live-server or notebook kernel):

In [2]:
file_download = pn.widgets.FileDownload(file='FileDownload.ipynb', filename='custom_filename.ipynb')

file_download

FileDownload(file='FileDownload.ipynb', filename='custom_filename.ipynb', label='Download custom_filename....)

The file data may also be embedded immediately using `embed` parameter, this allows using the widget even in a static export:

In [3]:
pn.widgets.FileDownload(file='FileDownload.ipynb', embed=True)

FileDownload(_transfers=1, data='data:application/octet-st..., embed=True, file='FileDownload.ipynb', filename='FileDownload.ipynb', label='Download FileDownload.ipy...)

If `auto=False` is set the file will not be downloaded on the initial click but will change the label from "Tranfer <file>" to "Download <file>" once the data has been synced. This offers an opportunity to download using the `Save as` dialog once the data has been transferred.

In [4]:
pn.widgets.FileDownload(
    file='FileDownload.ipynb', button_type='success', auto=False,
    embed=False, name="Right-click to download using 'Save as' dialog"
)

FileDownload(auto=False, button_type='success', file='FileDownload.ipynb', label='Transfer FileDownload.ipy..., name="Right-click t...)

The `FileDownload` widget may also be given a file-like object, e.g. here we save a pandas DataFrame as a CSV to a StringIO object and pass that to the widget:

In [5]:
from bokeh.sampledata.autompg import autompg

from io import StringIO
sio = StringIO()
autompg.to_csv(sio)
sio.seek(0)

pn.widgets.FileDownload(sio, embed=True, filename='autompg.csv')

FileDownload(_transfers=1, data='data:text/plain;charset=U..., embed=True, file=<_io.StringIO o..., filename='autompg.csv', label='Download autompg.csv')

If you want to generate the file dynamically, e.g. because it depends on the parameters of some widget you can also supply a callback (which may be decorated with the widgets and/or parameters it depends on):

In [6]:
years_options = list(autompg.yr.unique())
years = pn.widgets.MultiChoice(
    name='Years', options=years_options, value=[years_options[0]], margin=(0, 20, 0, 0)
)
mpg = pn.widgets.RangeSlider(
    name='Mile per Gallon', start=autompg.mpg.min(), end=autompg.mpg.max()
)

@pn.depends(years, mpg)
def filtered_mpg(yrs, mpg):
    df = autompg
    if years.value:
        df = autompg[autompg.yr.isin(yrs)]
    return df[(df.mpg >= mpg[0]) & (df.mpg <= mpg[1])]

@pn.depends(years, mpg)
def filtered_file(yr, mpg):
    df = filtered_mpg(yr, mpg)
    sio = StringIO()
    df.to_csv(sio)
    sio.seek(0)
    return sio

fd = pn.widgets.FileDownload(
    callback=filtered_file, filename='filtered_autompg.csv'
)

pn.Column(pn.Row(years, mpg), fd, pn.panel(filtered_mpg, width=600), width=600)

Column(width=600)
    [0] Row
        [0] MultiChoice(margin=(0, 20, 0, 0), name='Years', options=[70, 71, 72, 73, ...], value=[70])
        [1] RangeSlider(end=46.6, name='Mile per Gallon', start=9.0, value=(9.0, 46.6), value_end=46.6, value_start=9.0)
    [1] FileDownload(callback=<function filtered_file a..., filename='filtered_autompg.csv', label='Download filtered_autompg...)
    [2] ParamFunction(function, width=600)